In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks/ESG/ESG-NEW
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ESG/ESG-NEW
bert_model.py  esgbert_finetune.py  main.ipynb	parse_pdf.py  pdf_dir


In [ ]:
!pip install pdfminer.six
from parse_pdf import file2csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.6 MB/s eta 0:00:00


In [ ]:
import os

PDF_DIR = 'pdf_dir'
NATIONS = 'american', 'taiwan'
OUTPUT_DIR = 'output'
PARSED_DIR = os.path.join(OUTPUT_DIR, 'parsed')

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
if not os.path.exists(PARSED_DIR):
    os.mkdir(PARSED_DIR)

for nation in NATIONS:
    pdf_dir = os.path.join(PDF_DIR, nation)
    csv_output_dir = os.path.join(PARSED_DIR, nation)
    if not os.path.exists(csv_output_dir):
        os.mkdir(csv_output_dir)
    for pdf_name in os.listdir(pdf_dir):
        pdf_path = os.path.join(pdf_dir, pdf_name)
        print(pdf_path)
        file2csv(pdf_path, csv_output_dir)

pdf_dir/american/ikea_sustainability_report_fy22_57c0217c71.pdf
pdf_dir/american/Flexsteel 2022-ESG-Report.pdf
pdf_dir/taiwan/台灣-Mediatek(聯發科).pdf
pdf_dir/taiwan/台灣-Cathay Financial.pdf


In [ ]:
from bert_model import EsgBertPredict

LABLED_DIR = os.path.join(OUTPUT_DIR, 'labled')
if not os.path.exists(LABLED_DIR):
    os.mkdir(LABLED_DIR)

for nation in NATIONS:
    csv_dir = os.path.join(PARSED_DIR, nation)
    output_dir = os.path.join(LABLED_DIR, nation)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for csv_name in os.listdir(csv_dir):
        csv_path = os.path.join(csv_dir, csv_name)
        print(csv_path)
        esg_bert = EsgBertPredict(csv_path, output_dir)
        esg_bert.main()


output/parsed/american/ikea_sustainability_report_fy22_57c0217c71.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

100%|██████████| 78/78 [12:34<00:00,  9.67s/it]


output/parsed/taiwan/台灣-Cathay Financial.csv


 53%|█████▎    | 51/96 [04:36<03:53,  5.19s/it]

Epoch 50, count is 0


100%|██████████| 96/96 [08:42<00:00,  5.44s/it]
